In [296]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle

In [599]:
#!pip install matplotlib
#!pip install seaborn

In [600]:
##source: https://data.world/data-ny-gov/jtrr-tvq4

# Predicting Energy and Gas Savings
====================================================

# Part 3: Machine Learning Models

Welcome to the third part of the Jupyter notebook, where we'll delve into building and evaluating machine learning models for predicting annual energy and annual gas savings in residential homes. In this phase, we'll explore three different machine learning models: Linear Model, Random Forest, and Gradient Boosting. The objective is to identify the most effective model for the specific prediction task.

**Background Recap:**

As a quick recap, we began by cleaning and preparing the dataset, focusing on projects within the Home Performance with ENERGY STAR® Program in the State of New York (US) from 2007 to 2012. The dataset includes backcasted estimated modeled savings and normalized values from an open-source energy efficiency meter.

**Exploratory Data Analysis (EDA):**

In the previous section (Part 2), we performed Exploratory Data Analysis (EDA) to gain insights into the dataset. Now, armed with a deeper understanding of our data, we're ready to move on to building and evaluating machine learning models.

**Machine Learning Models:**

1. Linear Model:
For the two target columns (reporting energy and reporting gas), we'll initiate our modeling journey with a basic Linear Model. This will serve as our baseline for predictions, assuming a linear relationship between input features and the respective target variable.

3. Random Forest:
Next, we'll explore the Random Forest algorithm for each target column, a versatile ensemble learning method that leverages multiple decision trees to improve predictive accuracy and control overfitting.

4. Gradient Boosting:
Finally, we'll implement the Gradient Boosting algorithm, another ensemble method that builds a series of weak learners to create a strong predictive model. This technique often provides superior performance compared to individual models.

**Model Evaluation:**

For each model, we'll evaluate its performance using appropriate metrics, considering factors such as Mean Squared Error (MSE) or R-squared. This evaluation process will help us identify the model that best suits our prediction task.

Let's begin the exciting journey of building and assessing our machine learning models! 🤖📈

## Load data

In [142]:
data = pd.read_csv('/Users/simonefischer/Ironhack/Week_9/Final_Project/data/cleaned_data.csv')

In [143]:
data.head()

,Unnamed: 0,project_id,contractor_id,project_county,project_city,project_zip,climate_zone,weather_station,weather_station-normalization,project_completion_date,...,consolidated_edison,lipa,national_grid,national_fuel_gas,nyseg,orange_and_rockland,rochester_gas_and_electric,all_false,coordinates,age_category
0,0,P00000034473,CY0000000014,Onondaga,Fabius,13063,5,725190,725190,2007-08-17,...,0,0,1,0,0,0,0,False,"42.850323, -75.979919",Moderate
1,1,P00000110370,CY0000000014,Onondaga,Nedrow,13120,5,725190,725190,2007-10-04,...,0,0,1,0,0,0,0,False,"42.950373, -76.163321",Very Old
2,2,P00000182080,CY0000000014,Onondaga,Jamesville,13078,5,725190,725190,2008-02-27,...,0,0,1,0,0,0,0,False,"42.976691, -76.069719",Moderate
3,3,P00000196191,CY0000000261,Albany,Albany,12203,5,725180,725180,2008-02-20,...,0,0,1,0,0,0,0,False,"42.680815, -73.836193",Old
4,4,P00000327900,CY0000000004,Erie,Buffalo,14221,5,725280,725280,2008-06-18,...,0,0,1,0,0,0,0,False,"42.980424, -78.728009",Moderate


In [144]:
data = data.drop(columns = ['Unnamed: 0', 'all_false'])

In [145]:
data.columns

Index(['project_id', 'contractor_id', 'project_county', 'project_city',
       'project_zip', 'climate_zone', 'weather_station',
       'weather_station-normalization', 'project_completion_date',
       'customer_type', 'size_of_home', 'volume_of_home', 'number_of_units',
       'year_home_built', 'total_project_cost', 'contractor_incentive',
       'total_incentive', 'amount_financed_through_program',
       'estimated_first_year_energy_bill_savings', 'baseline_electric',
       'baseline_gas', 'reporting_electric', 'reporting_gas',
       'evaluated_annual_electric_savings', 'evaluated_annual_gas_savings',
       'central_hudson', 'consolidated_edison', 'lipa', 'national_grid',
       'national_fuel_gas', 'nyseg', 'orange_and_rockland',
       'rochester_gas_and_electric', 'coordinates', 'age_category'],
      dtype='object')

# Target columns

In [401]:
##The idea is to check for the success of the project by predicting the 'reporting_electric' and the 'reporting_gas'

In [402]:
##No need to check if my columns to be predicted are balanced because both future target variable is a continuous variable with a large range of unique values

In [146]:
data['reporting_electric'].nunique()

2641

In [147]:
data['reporting_gas'].nunique()

1923

# X/y split

The final selected columns for a linear regression are:  ['climate_zone', 'weather_station', 'weather_station-normalization', 'size_of_home', 'number_of_units', 'year_home_built', 'total_project_cost', 'contractor_incentive', 'total_incentive', 'amount_financed_through_program', 'baseline_electric', 'baseline_gas', 'central_hudson', 'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland', 'rochester_gas_and_electric']

--> a linear regression model will most likely not make any sense anyways because the columns are not correlated

--> I will further exclude weather_station and just include weather_station-normalization

# 1. Prediting Reporting Gas

# 1.1. Linear Regression

In [287]:
# Define feature columns and target column
feature_columns = [
    'climate_zone', 'weather_station', 'size_of_home', 'number_of_units', 'year_home_built',
    'total_project_cost', 'contractor_incentive', 'total_incentive', 'amount_financed_through_program',
     'baseline_gas']

target_column = "reporting_gas"

# Create the feature matrix (X) and target variable (y)
X = data[feature_columns]
y = data[target_column]


In [288]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [289]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [290]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [291]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [292]:
y_pred = model.predict(X_test_scaled)

In [293]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 317.09977752583046
R-squared: 0.949406783526552


In [294]:
r2_train = model.score(X_train_scaled, y_train)
r2_test = model.score(X_test_scaled, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.92
The R2 of the model in the TEST set is: 0.95


In [298]:
file = "linear_reg_reporting_gas_model.pkl"

with open(file, "wb") as file:
    pickle.dump(model, file)

# 1.2 Random forest

In [299]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming you have already loaded your cleaned_data DataFrame

# Split the data into features (X) and target variable (y)
y = data['reporting_gas']
X = data[['climate_zone', 'weather_station', 'weather_station-normalization', 'size_of_home', 'number_of_units', 'year_home_built', 'total_project_cost', 'contractor_incentive', 'total_incentive', 'amount_financed_through_program', 'baseline_electric', 'baseline_gas', 'central_hudson', 'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland', 'rochester_gas_and_electric']]


# Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Combine training and test sets for categorical features
combined_cat_data = pd.concat([X_train.select_dtypes(object), X_test.select_dtypes(object)])

# Encode categorical features
levels = [list(combined_cat_data[col].unique()) for col in combined_cat_data.columns]
encoder = OneHotEncoder(drop='first', categories=levels).fit(combined_cat_data)

X_train_cat_encoded_np = encoder.transform(X_train.select_dtypes(object)).toarray()
X_test_cat_encoded_np = encoder.transform(X_test.select_dtypes(object)).toarray()

X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

# Separate numeric and encoded categorical features
X_train_num = X_train.select_dtypes(np.number)
X_test_num = X_test.select_dtypes(np.number)

# Concatenate numeric and encoded categorical features
X_train = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [300]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 295.02204558145
R-squared: 0.9068421119288239


In [301]:
r2_train = regressor.score(X_train, y_train)
r2_test = regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.98
The R2 of the model in the TEST set is: 0.91


In [302]:
file = "random_forest_reporting_gas_model.pkl"

with open(file, "wb") as file:
    pickle.dump(regressor, file)

### Trying to define better hyperparameters

In [70]:
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestRegressor()

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [80, 100, 120],  
    'min_samples_split': [2, 3],  
    'min_samples_leaf': [1, 2], 
    'max_depth': [5, 10],
    'max_features': [0.8, 1.0]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5,return_train_score=True,n_jobs=1, verbose = 0)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_


In [71]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=10, min_samples_leaf=2, n_estimators=80)

In [78]:
##fitting the model with the estimated best hyperparameters

regressor_hyper = RandomForestRegressor(max_depth=10, min_samples_leaf=2, n_estimators=80)

# Train the model
regressor_hyper.fit(X_train, y_train)

# Make predictions on the test set
predictions = regressor_hyper.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1323808.6550104148
R-squared: 0.7735800709882532


The model seems to perfom best with the default settings.

### Feature Importances

after trying around with hyperparameters that are close to the default, the default seems to perform slightly better so I will continue with the default values

In [188]:
# Get feature importances
feature_importances = regressor.feature_importances_

# Create a Series with feature importances, indexed by feature names
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# Sort the Series by importance in descending order
feature_importance_series = feature_importance_series.sort_values(ascending=False)

# Print the top features
num_top_features = 20  
print("Top features:")
print(feature_importance_series.head(num_top_features))


Top features:
baseline_gas                       0.954353
size_of_home                       0.010907
total_incentive                    0.007183
year_home_built                    0.006339
total_project_cost                 0.004536
amount_financed_through_program    0.003621
contractor_incentive               0.003536
baseline_electric                  0.003138
weather_station                    0.002308
weather_station-normalization      0.002056
climate_zone                       0.000553
nyseg                              0.000487
national_fuel_gas                  0.000354
orange_and_rockland                0.000221
consolidated_edison                0.000154
number_of_units                    0.000151
rochester_gas_and_electric         0.000092
central_hudson                     0.000009
lipa                               0.000000
dtype: float64


### Simple linear regression to just check on baseline_gas

baseline_gas seems by far the most important feature I will try as a last step to do a linear regression only with baseline_gas

In [190]:
X = data[['baseline_gas']]
y = data['reporting_gas']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 323.49554277478194
R-squared: 0.94838634025069


In [191]:
r2_train = model.score(X_train_scaled, y_train)
r2_test = model.score(X_test_scaled, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.91
The R2 of the model in the TEST set is: 0.95


The linear regression with more features performes still better.

# 1.3 Gradient Boosting

In [187]:
from sklearn.ensemble import GradientBoostingRegressor

# Create and fit the GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Make predictions
y_train_pred = gb_regressor.predict(X_train)
y_test_pred = gb_regressor.predict(X_test)

# Evaluate the model
r2_train = gb_regressor.score(X_train, y_train)
r2_test = gb_regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))


The R2 of the model in the TRAIN set is: 0.97
The R2 of the model in the TEST set is: 0.91


In [304]:
file = "gradient_boosting_reporting_gas_model.pkl"

with open(file, "wb") as file:
    pickle.dump(gb_regressor, file)

### Trying to define better hyperparameters

In [82]:
param_grid = {
    'n_estimators': [80, 100, 120],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 3], 
    'max_depth': [3, 5, 7],  
    'max_features': ['sqrt'], 
    'learning_rate': [0.05, 0.1, 0.2] 
}

gb = GradientBoostingRegressor(random_state=100)

grid_search = GridSearchCV(gb, param_grid, cv=5,return_train_score=True,n_jobs=1, verbose = 0)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned

{'learning_rate': 0.2,
 'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 120}

In [83]:
grid_search.best_estimator_

GradientBoostingRegressor(learning_rate=0.2, max_depth=7, max_features='sqrt',
                          min_samples_leaf=3, n_estimators=120,
                          random_state=100)

In [84]:
from sklearn.model_selection import cross_val_score

gb = GradientBoostingRegressor(learning_rate=0.2, max_depth=7, max_features='sqrt',
                          min_samples_leaf=3, n_estimators=120,
                          random_state=100)

cross_val_scores = cross_val_score(gb, X_train, y_train, cv=10)
print("The mean R2 of over the folds was {:.2f}".format(np.mean(cross_val_scores)))

The mean R2 of over the folds was 0.53


In [85]:
cross_val_scores

array([0.65324378, 0.49393291, 0.59835411, 0.24824606, 0.5624993 ,
       0.28373649, 0.73823369, 0.60170852, 0.63458959, 0.44261176])

In [86]:
gb.fit(X_train, y_train)
print("The R2 for the model in the TRAIN set is {:.3f}".format(gb.score(X_train,y_train)))
print("The R2 for the model in the TRAIN set is {:.3f}".format(gb.score(X_test,y_test)))

The R2 for the model in the TRAIN set is 0.898
The R2 for the model in the TRAIN set is 0.557


The model seems to perfom best with the default settings.

# 1.4 Conclusions

The linear regression model seems to perfom best in order to predict to reporting gas.

**X and y data:**

feature_columns = [
    'climate_zone', 'weather_station', 'size_of_home', 'number_of_units', 'year_home_built',
    'total_project_cost', 'contractor_incentive', 'total_incentive', 'amount_financed_through_program',
     'baseline_gas']

target_column = "reporting_gas"

X = data[feature_columns]

y = data[target_column]

**Error metrics:**

Mean Squared Error: 317.09977752583046

R-squared: 0.949406783526552

The R2 of the model in the TRAIN set is: 0.92

The R2 of the model in the TEST set is: 0.95

# 2. Predicting the reporting electric

# 2.1 Linear Regression

In [305]:
# Define feature columns and target column
feature_columns = ['climate_zone', 'weather_station', 'weather_station-normalization', 'size_of_home', 'number_of_units', 'year_home_built', 'total_project_cost', 'contractor_incentive', 'total_incentive', 'amount_financed_through_program', 'baseline_electric', 'central_hudson', 'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland', 'rochester_gas_and_electric']

target_column = "reporting_electric"

# Create the feature matrix (X) and target variable (y)
X = data[feature_columns]
y = data[target_column]


In [306]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [307]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [308]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [309]:
y_pred = model.predict(X_test_scaled)

In [310]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 3416857.1095308335
R-squared: 0.8877479461244696


In [213]:
np.sqrt(3597269.097657373)

1896.6468036135177

In [311]:
r2_train = model.score(X_train_scaled, y_train)
r2_test = model.score(X_test_scaled, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.87
The R2 of the model in the TEST set is: 0.89


In [312]:
file = "linear_reg_reporting_electric_model.pkl"

with open(file, "wb") as file:
    pickle.dump(model, file)

# 2.2 Random Forest

In [313]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming you have already loaded your cleaned_data DataFrame

# Split the data into features (X) and target variable (y)
y = data['reporting_electric']
X = data[
    ['climate_zone', 'weather_station', 'weather_station-normalization', 'size_of_home', 'number_of_units',
     'year_home_built', 'total_project_cost', 'contractor_incentive', 'total_incentive',
     'amount_financed_through_program', 'baseline_electric', 'baseline_gas', 'central_hudson', 'consolidated_edison',
     'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland', 'rochester_gas_and_electric']
]

# Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Combine training and test sets for categorical features
combined_cat_data = pd.concat([X_train.select_dtypes(object), X_test.select_dtypes(object)])

# Encode categorical features
levels = [list(combined_cat_data[col].unique()) for col in combined_cat_data.columns]
encoder = OneHotEncoder(drop='first', categories=levels).fit(combined_cat_data)

X_train_cat_encoded_np = encoder.transform(X_train.select_dtypes(object)).toarray()
X_test_cat_encoded_np = encoder.transform(X_test.select_dtypes(object)).toarray()

X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

# Separate numeric and encoded categorical features
X_train_num = X_train.select_dtypes(np.number)
X_test_num = X_test.select_dtypes(np.number)

# Concatenate numeric and encoded categorical features
X_train = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [314]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 2557843.632673517
R-squared: 0.9032919163504649


In [315]:
r2_train = regressor.score(X_train, y_train)
r2_test = regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.98
The R2 of the model in the TEST set is: 0.90


In [316]:
file = "random_forest_reporting_electric_model.pkl"

with open(file, "wb") as file:
    pickle.dump(regressor, file)

### Feature Importances

In [241]:
# Get feature importances
feature_importances = regressor.feature_importances_

# Create a Series with feature importances, indexed by feature names
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# Sort the Series by importance in descending order
feature_importance_series = feature_importance_series.sort_values(ascending=False)

# Print the top features
num_top_features = 10  
print("Top features:")
print(feature_importance_series.head(num_top_features))

Top features:
baseline_electric                  0.915472
size_of_home                       0.015509
year_home_built                    0.011651
total_project_cost                 0.011023
weather_station                    0.010915
total_incentive                    0.009057
weather_station-normalization      0.008275
contractor_incentive               0.005850
amount_financed_through_program    0.004870
baseline_gas                       0.004247
dtype: float64


# 2.3 Gradient Boosting

In [319]:
from sklearn.ensemble import GradientBoostingRegressor

# Create and fit the GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Make predictions
y_train_pred = gb_regressor.predict(X_train)
y_test_pred = gb_regressor.predict(X_test)

# Evaluate the model
r2_train = gb_regressor.score(X_train, y_train)
r2_test = gb_regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))


The R2 of the model in the TRAIN set is: 0.94
The R2 of the model in the TEST set is: 0.89


In [320]:
file = "gradient_boosting_reporting_electric_model.pkl"

with open(file, "wb") as file:
    pickle.dump(gb_regressor, file)

### Trying to define better hyperparameters

In [97]:
param_grid = {
    'n_estimators': [80, 100, 120],  # Close to the default value
    'min_samples_split': [2, 5, 10],  # Close to the default value
    'min_samples_leaf': [1, 2, 3],  # Close to the default value
    'max_depth': [3, 5, 7],  # Close to the default value
    'max_features': ['sqrt'],  # Close to the default value
    'learning_rate': [0.05, 0.1, 0.2]  # Adjusted based on default value
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=5, n_jobs=1, verbose=0)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

In [98]:
grid_search.best_estimator_

GradientBoostingRegressor(learning_rate=0.2, max_depth=7, max_features='sqrt',
                          min_samples_leaf=3, min_samples_split=5,
                          n_estimators=120)

In [243]:
from sklearn.model_selection import cross_val_score

gb = GradientBoostingRegressor(learning_rate=0.2, max_depth=7, max_features='sqrt',
                          min_samples_leaf=3, min_samples_split=5,
                          n_estimators=150)

cross_val_scores = cross_val_score(gb, X_train, y_train, cv=10)
print("The mean R2 of over the folds was {:.2f}".format(np.mean(cross_val_scores)))

The mean R2 of over the folds was 0.86


In [244]:
cross_val_scores

array([0.84842031, 0.86958734, 0.80853867, 0.80915785, 0.87858098,
       0.89714313, 0.86828966, 0.91616376, 0.88417729, 0.85915598])

In [245]:
gb.fit(X_train, y_train)
print("The R2 for the model in the TRAIN set is {:.3f}".format(gb.score(X_train,y_train)))
print("The R2 for the model in the TRAIN set is {:.3f}".format(gb.score(X_test,y_test)))

The R2 for the model in the TRAIN set is 0.994
The R2 for the model in the TRAIN set is 0.883


With other hyperparameters, the models seems to overfit more.

# 2.4 Conclusions

The random forest model seems to perfom best in order to predict the reporting electric.

**X and y data:**

X = data[
    ['climate_zone', 'weather_station', 'weather_station-normalization', 'size_of_home', 'number_of_units',
     'year_home_built', 'total_project_cost', 'contractor_incentive', 'total_incentive',
     'amount_financed_through_program', 'baseline_electric', 'baseline_gas', 'central_hudson', 'consolidated_edison',
     'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland', 'rochester_gas_and_electric']
]

y = data['reporting_electric']

**Error metrics:**

Mean Squared Error: 2557843.632673517

R-squared: 0.9032919163504649

The R2 of the model in the TRAIN set is: 0.98

The R2 of the model in the TEST set is: 0.90

**Most important features (proportion of total impurity):**

- baseline_electric                  0.915472
- size_of_home                       0.015509
- year_home_built                    0.011651
- total_project_cost                 0.011023
- weather_station                    0.010915

# 3. Predicting the total project cost

# 3.1 Linear Regression

In [321]:
feature_columns2 = ['climate_zone', 'weather_station-normalization', 'size_of_home', 'number_of_units',
                    'year_home_built', 'contractor_incentive', 'total_incentive',
                    'amount_financed_through_program', 'baseline_gas', 'baseline_electric', 'reporting_electric', 'reporting_gas', 'central_hudson',
                    'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland',
                    'rochester_gas_and_electric']

target_column2 = "total_project_cost"

# Create the feature matrix (X) and target variable (y)
X = data[feature_columns2]
y = data[target_column2]

In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [323]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [324]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [325]:
y_pred = model.predict(X_test_scaled)

In [326]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 9354795.161353061
R-squared: 0.5769279496672461


In [327]:
r2_train = model.score(X_train_scaled, y_train)
r2_test = model.score(X_test_scaled, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.61
The R2 of the model in the TEST set is: 0.58


In [328]:
file = "linear_reg_total_cost_model.pkl"

with open(file, "wb") as file:
    pickle.dump(model, file)

# 3.2 Random Forest

In [329]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming you have already loaded your cleaned_data DataFrame

# Split the data into features (X) and target variable (y)
y = data['total_project_cost']
X = data[['climate_zone', 'weather_station-normalization', 'size_of_home', 'number_of_units',
                    'year_home_built', 'contractor_incentive', 'total_incentive',
                    'amount_financed_through_program', 'baseline_gas', 'baseline_electric', 'reporting_electric', 'reporting_gas', 'central_hudson',
                    'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland',
                    'rochester_gas_and_electric']]

# Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Combine training and test sets for categorical features
combined_cat_data = pd.concat([X_train.select_dtypes(object), X_test.select_dtypes(object)])

# Encode categorical features
levels = [list(combined_cat_data[col].unique()) for col in combined_cat_data.columns]
encoder = OneHotEncoder(drop='first', categories=levels).fit(combined_cat_data)

X_train_cat_encoded_np = encoder.transform(X_train.select_dtypes(object)).toarray()
X_test_cat_encoded_np = encoder.transform(X_test.select_dtypes(object)).toarray()

X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

# Separate numeric and encoded categorical features
X_train_num = X_train.select_dtypes(np.number)
X_test_num = X_test.select_dtypes(np.number)

# Concatenate numeric and encoded categorical features
X_train = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [330]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1704874.0481009507
R-squared: 0.9205169893134203


In [331]:
r2_train = regressor.score(X_train, y_train)
r2_test = regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.98
The R2 of the model in the TEST set is: 0.92


In [332]:
file = "random_forest_total_cost_model.pkl"

with open(file, "wb") as file:
    pickle.dump(regressor, file)

### Feature Importances

In [285]:
# Get feature importances
feature_importances = regressor.feature_importances_

# Create a Series with feature importances, indexed by feature names
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# Sort the Series by importance in descending order
feature_importance_series = feature_importance_series.sort_values(ascending=False)

# Print the top features
num_top_features = 10  
print("Top features:")
print(feature_importance_series.head(num_top_features))

Top features:
contractor_incentive               0.698452
total_incentive                    0.157025
amount_financed_through_program    0.070754
size_of_home                       0.018144
reporting_electric                 0.011872
year_home_built                    0.010239
baseline_electric                  0.007498
climate_zone                       0.007264
weather_station-normalization      0.006663
reporting_gas                      0.003146
dtype: float64


# 3.3 Gradient Boosting

In [333]:
from sklearn.ensemble import GradientBoostingRegressor

# Create and fit the GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Make predictions
y_train_pred = gb_regressor.predict(X_train)
y_test_pred = gb_regressor.predict(X_test)

# Evaluate the model
r2_train = gb_regressor.score(X_train, y_train)
r2_test = gb_regressor.score(X_test, y_test)

print("The R2 of the model in the TRAIN set is: {:.2f}".format(r2_train))
print("The R2 of the model in the TEST set is: {:.2f}".format(r2_test))

The R2 of the model in the TRAIN set is: 0.93
The R2 of the model in the TEST set is: 0.91


In [334]:
file = "gradient_boosting_total_cost_model.pkl"

with open(file, "wb") as file:
    pickle.dump(gb_regressor, file)

# 3.4 Conclusions

The random forest model seems to perfom best in order to predict the total project cost.

**X and y data:**

X = data[['climate_zone', 'weather_station-normalization', 'size_of_home', 'number_of_units',
                    'year_home_built', 'contractor_incentive', 'total_incentive',
                    'amount_financed_through_program', 'baseline_gas', 'baseline_electric', 'reporting_electric', 'reporting_gas', 'central_hudson',
                    'consolidated_edison', 'lipa', 'national_fuel_gas', 'nyseg', 'orange_and_rockland',
                    'rochester_gas_and_electric']]

y = data['total_project_cost']


**Error metrics:**

Mean Squared Error: 1704874.0481009507

R-squared: 0.9205169893134203

The R2 of the model in the TRAIN set is: 0.98

The R2 of the model in the TEST set is: 0.92

**Most important features (proportion of total impurity):**

- contractor_incentive               0.698452
- total_incentive                    0.157025
- amount_financed_through_program    0.070754
- size_of_home                       0.018144
- reporting_electric                 0.011872
